In [ ]:
# 本代码由可视化策略环境自动生成 2021年12月21日 22:26
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


# Python 代码入口函数，input_1/2/3 对应三个输入端，data_1/2/3 对应三个输出端
def m1_run_bigquant_run(input_1, input_2, input_3):
    df = input_1.read_pickle()  # 日期
        
    start_date = pd.to_datetime(df['start_date'])
    end_date = pd.to_datetime(df['end_date'])
    
    bm_instruments = ['000300.HIX']
    start_date = start_date - datetime.timedelta(20)
    id_ohlc = 'bar1d_index_CN_STOCK_A'
    origin_fields=['open', 'high', 'low', 'close', 'volume', 'amount']
    df = DataSource(id_ohlc).read(instruments=bm_instruments, start_date=start_date, end_date=end_date, fields=origin_fields)
    df.rename({'s_dq_close':'close', 's_dq_high':'high', 's_dq_open':'open', 's_dq_low':'low', 's_dq_volume':'volume', 's_dq_amount':'amount'},axis=1, inplace=True)
    df['pct5'] = df['close'].pct_change(5)
    df.dropna(inplace=True)
    df['bm'] = df.apply(lambda x: 1 if x['pct5']<-0.05 else 0, axis=1)
    
    benchmark_ds = DataSource.write_df(df)
    return Outputs(data_1=benchmark_ds, data_2=None, data_3=None)

# 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
def m1_post_run_bigquant_run(outputs):
    return outputs

def m2_generate_sql_bigquant_run():
    # 使用sql读取数据，数量限制在10000行内
    sql = "SELECT a.ts_code as instrument, a.trade_date as date, a.open, a.high, a.low, a.close, a.vol as volume, a.amount, b.market_name, b.ts_code as index_code, b.w_score, b.bias_cs, b.bias_sm, b.bias_ml, b.bias_cs_dd, b.bias_sm_dd, b.bias_ml_dd, b.cs_diff1, b.sm_diff1, b.ml_diff1, b.cs_diff2, b.sm_diff2, b.ml_diff2, c.adj_factor FROM 嘉翼基金池日线数据 as a, 嘉翼1号指数基金宽度数据 as b, 嘉翼基金池复权因子 as c where b.trade_ts_code = a.ts_code and a.ts_code = c.ts_code and a.trade_date = b.trade_date and a.trade_date = c.trade_date and a.trade_date >= '2017-01-01' order by date"  #  and a.trade_date <= '2019-06-01'
#    sql = "SELECT a.ts_code as instrument, a.trade_date as date, a.open, a.high, a.low, a.close, b.market_name, b.ts_code as index_code, b.w_score, b.bias_cs, b.bias_sm, b.bias_ml, b.bias_cs_dd, b.bias_sm_dd, b.bias_ml_dd, b.cs_diff1, b.sm_diff1, b.ml_diff1, b.cs_diff2, b.sm_diff2, b.ml_diff2 FROM 嘉翼基金池日线数据 as a, 嘉翼1号指数基金宽度数据 as b where b.trade_ts_code = a.ts_code and a.trade_date = b.trade_date and a.trade_date >= '2019-01-01' and a.ts_code in ('159928.SZ','515050.SH','510330.SH','510500.SH','512010.SH','512400.SH','512690.SH','512680.SH','159915.SZ') order by date"  #  and a.trade_date <= '2019-06-01'
    return sql
    # return ""

# Python 代码入口函数，input_1/2/3 对应三个输入端，data_1/2/3 对应三个输出端
def m4_run_bigquant_run(input_1, input_2, input_3):
    # 示例代码如下。在这里编写您的代码
    df = input_1.read_df()
    df.to_hdf('pq_data.h5', key='date')
    data_1 = DataSource.write_df(df)
    return Outputs(data_1=None, data_2=None, data_3=None)

# 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
def m4_post_run_bigquant_run(outputs):
    return outputs

# Python 代码入口函数，input_1/2/3 对应三个输入端，data_1/2/3 对应三个输出端
def m5_run_bigquant_run(input_1, input_2, input_3):
    import pandas as pd
    import postgresql
    from psycopg2 import connect, DatabaseError

    param_dic = {
        "host": "121.41.121.43",
        "database": "jyquant",
        "user": "postgres",
        "password": "Cong2021@DB009",
        "port": "5433"
    }

    def create_connect(param_dic=None, debug=False):
        """
        基于配置表建立一个与数据库的联接，返回 conn

        :param params_dic:
        :param debug:
        :return:
        """
        try:
            conn = connect(**param_dic)
        except (Exception, DatabaseError) as error:
            print('数据库连接失败，异常：', error)
        if debug:
            print('数据库连接成功')
        return conn

    def end_connect(conn=None, debug=False):
        """
        结束与数据库的联接

        :param conn:
        :param debug:
        :return:
        """
        if conn is not None:
            try:
                conn.commit()
                conn.close()
            except (Exception, DatabaseError) as error:
                print('数据库关闭失败，异常：', error)
            if debug:
                print('数据库连接关闭')

    def execute_query(con=None, query=None):
        """
        根据条件创建一个更新语句

        :param con:
        :param query:
        :return:
        """
        cursor = con.cursor()
        try:
            cursor.execute(query)
            con.commit()
        except (Exception, DatabaseError) as error:
            print("执行时出错: %s" % error)
            con.rollback()
            cursor.close()
            return True
        cursor.close()

    db_conn = create_connect(param_dic=param_dic, debug=True)

    query = f'select * from 嘉翼1号基金池'

    df = execute_query(conn=db_conn, query=query)
    end_connect(conn=db_conn, debug=True)

    data_1 = DataSource.write_df(df)
    return Outputs(data_1=data_1, data_2=None, data_3=None)

# 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
def m5_post_run_bigquant_run(outputs):
    return outputs

# 策略容量计算函数
def m11_calculate_bigquant_run(results, volume_limit, frequency, output='display'):
    import numpy as np
    import pandas as pd
    from empyrical import alpha_beta_aligned, annual_return, annual_volatility, cum_returns, max_drawdown, sharpe_ratio

    risk_free_interest = 0.0001173

    def get_max_drawdown(returns):
        """计算最大回撤"""
        returns = returns.copy()
        df_cum = cum_returns(returns, 1.0)
        running_max = np.maximum.accumulate(df_cum)
        underwater = df_cum / running_max - 1
        valley = underwater.index[np.argmin(underwater)]
        peak = underwater[:valley][underwater[:valley] == 0].index[-1]
        try:
            recovery = underwater[valley:][underwater[valley:] == 0].index[0]
        except IndexError:
            recovery = np.nan
        return peak, valley, recovery

    def sum_buy_and_sell_turnover(transactions):
        """统计每日交易额"""
        total_turnover = 0.0
        for t in transactions:
            total = abs(t['amount'] * t['price'])
            commission = t['commission']
            total_turnover += total + commission
        return total_turnover

    def prepare_results(results):
        """处理传入的回测数据，计算出后续需要用到的数据"""
        # 策略累计收益率
        algorithm_cumulative_returns = results['algorithm_period_return'] + 1.0
        algorithm_cumulative_returns_shift = results['algorithm_period_return'].shift(1).fillna(0) + 1.0
        # 策略日收益率
        results['algorithm_simple_returns'] = algorithm_cumulative_returns / algorithm_cumulative_returns_shift - 1.0
        # 基准累计收益率
        benchmark_cumulative_returns = results['benchmark_period_return'] + 1.0
        benchmark_cumulative_returns_shift = results['benchmark_period_return'].shift(1).fillna(0) + 1.0
        # 基准日收益率
        results['benchmark_simple_returns'] = benchmark_cumulative_returns / benchmark_cumulative_returns_shift - 1.0
        # 超额收益
        results['excess_simple_returns'] = results['algorithm_simple_returns'] - results['benchmark_simple_returns']
        # 单利累计相对收益
        results['excess_cumulative_returns'] = results['excess_simple_returns'].cumsum()
        # 单利累计相对收益百分比
        results['excess_cumulative_ratio'] = results['excess_cumulative_returns']
        return results

    def get_stats(results):
        """计算收益相关信息"""
        excess_return_ratio = round(results['excess_cumulative_ratio'].iloc[-1] * 100, 2)
        annual_return_ratio = round(annual_return(results['excess_simple_returns']) * 100, 2)
        alpha, beta = alpha_beta_aligned(
            results['excess_simple_returns'],
            results['benchmark_simple_returns'],
            risk_free_interest
        )
        sharp_ratio = round(
            sharpe_ratio(
                results['excess_simple_returns'],
                risk_free_interest
            ), 2
        )
  

        if sharp_ratio < -1.0e+6:
            sharp_ratio = 'n/a'

        # 收益波动率
        return_volatility = round(annual_volatility(results['excess_simple_returns']) * 100, 2)
        # 最大回撤和最大回撤天数
        max_drawdown_value = round(abs(max_drawdown(results['excess_simple_returns']) * 100), 2)
        peak, valley, recovery = get_max_drawdown(results['excess_simple_returns'])
        max_drawdown_days = abs((peak - valley).days)

        # 计算平均每日换手率
        daily_transaction = results['transactions'].apply(sum_buy_and_sell_turnover)
        daily_portfolio_value = results['portfolio_value'].shift(1)
        daily_turnover_ratio = daily_transaction / daily_portfolio_value
        average_daily_turnover = round(daily_turnover_ratio.mean() * 100, 2)

        ret = {
            'excess_return_ratio': excess_return_ratio,
            'annual_return_ratio': annual_return_ratio,
            'beta': round(beta, 2),
            'alpha': round(alpha, 2),
            'sharp_ratio': sharp_ratio,
            'return_volatility': return_volatility,
            'max_drawdown': max_drawdown_value,
            'max_drawdown_days': max_drawdown_days,
            'average_daily_turnover': average_daily_turnover,
        }
        return ret

    def plot_excess_cumulative_ratio(results, output='script'):
        """组合单利相对收益净值作图"""
        options = {
            'legend': {
                'enabled': True,
            },
            'plotOptions': {
                'series': {
                    'showInNavigator': True,
                },
            },
            'series': [
                {
                    'name': '组合单利相对收益净值',
                    'type': 'spline',
                    'tooltip': {
                        'valueDecimals': 2,
                        'valueSuffix': ' ',
                    },
                },
            ],
            'yAxis': [
                {
                    'title': {
                        'text': '净值',
                    },
                    'labels': {
                        'format': '{value}'
                    },
                },
            ]
        }
        return T.plot(results['excess_cumulative_ratio'] + 1, options=options, output=output)

    def plot_strategy_capacity(results, volume_limit, frequency, output='script'):
        """计算策略容量并作图"""
        # 持仓详情
        positions = results['positions']
        # 获取回测开始时间和结束时间
        start_date = results.index.min().strftime('%Y-%m-%d')
        end_date = results.index.max().strftime('%Y-%m-%d')
        # 获取开盘 N 分钟行情数据
        frequency_cols_map = {
            '15m': ['wap_2_buy_volume', 'wap_2_twap_buy'],
            '30m': ['wap_3_buy_volume', 'wap_3_twap_buy'],
            '60m': ['wap_4_buy_volume', 'wap_4_twap_buy'],
        }
        rename_dict = {
            '15m': {
                'wap_2_buy_volume': 'volume',
                'wap_2_twap_buy': 'twap',
            },
            '30m': {
                'wap_3_buy_volume': 'volume',
                'wap_3_twap_buy': 'twap',
            },
            '60m': {
                'wap_4_buy_volume': 'volume',
                'wap_4_twap_buy': 'twap',
            },
        }
        bar_data = DataSource('bar1d_wap_CN_STOCK_A').read(start_date=start_date, end_date=end_date, fields=frequency_cols_map.get(frequency))
        bar_data.rename(columns=rename_dict.get(frequency), inplace=True)
        bar_data['amount'] = bar_data['volume'] * bar_data['twap']
        bar_data.set_index('date', inplace=True)
        # 时间序列
        date_range = np.unique(positions.index.drop_duplicates().strftime('%Y-%m-%d'))
        all_strategy_capacity = []
        for d in date_range:
            daily_positions = [p['sid'].symbol for p in positions[d][0]]
            if not daily_positions:
                # 当天无持仓则跳过，容量值按 0 处理
                all_strategy_capacity.append(0)
                continue
            daily_data = bar_data[bar_data.index == d]
            condition = daily_data['instrument'].isin(daily_positions) & (daily_data['volume'] != 0)
            daily_instrument = daily_data[condition].reset_index()[['instrument', 'amount']].to_dict('records')
            if not daily_instrument:
                # 未找到对应股票则跳过当天，容量值按 0 处理
                all_strategy_capacity.append(0)
                continue
            min_daily_instrument = min(daily_instrument, key=lambda x: x.get('amount'))
            min_daily_amount = min_daily_instrument.get('amount', 0)
            daily_strategy_capacity = min_daily_amount * volume_limit * len(daily_positions)
            all_strategy_capacity.append(daily_strategy_capacity)
        # 策略容量
        strategy_capacity_df = pd.DataFrame(data=all_strategy_capacity, columns=['strategy_capacity'],
                                            index=results.index)
        # 总资产
        strategy_capacity_df['portfolio_value'] = results['portfolio_value']
        options = {
            'title': {
                'text': ''
            },
            'subtitle': {
                'text': ''
            },
            'legend': {
                'enabled': True,
            },
            'plotOptions': {
                'series': {
                    'showInNavigator': True,
                },
            },
            'yAxis': [
                {
                    'labels': {
                        'format': '{value}',
                    },
                    'title': {
                        'text': '总资产/策略容量',
                    },
                }
            ],
            'series': [
                {
                    'name': '策略容量',
                    'type': 'column',
                    'yAxis': 0,
                    'tooltip': {
                        'valueSuffix': ' '
                    }
                },
                {
                    'name': '总资产',
                    'type': 'spline',
                    'color': '#ff3300',
                    'yAxis': 0,
                    'tooltip': {
                        'valueDecimals': 2,
                        'valueSuffix': ' '
                    }
                }
            ]
        }
        return T.plot(strategy_capacity_df, options=options, output=output)

    results = prepare_results(results)
    stats = get_stats(results)
    stats_html = f"""
    <div class='kpicontainer'>
        <ul class='kpi'>
            <li><span class='title'>超额收益率</span><span class='value'>{stats['excess_return_ratio']}%</span></li>
            <li><span class='title'>超额年化收益率</span><span class='value'>{stats['annual_return_ratio']}%</span></li>
            <li><span class='title'>阿尔法</span><span class='value'>{stats['alpha']}</span></li>
            <li><span class='title'>贝塔</span><span class='value'>{stats['beta']}</span></li>
            <li><span class='title'>夏普比率</span><span class='value'>{stats['sharp_ratio']}</span></li>
            <li><span class='title'>收益波动率</span><span class='value'>{stats['return_volatility']}%</span></li>
            <li><span class='title'>最大回撤</span><span class='value'>{stats['max_drawdown']}%</span></li>
            <li><span class='title'>最大回撤天数</span><span class='value'>{stats['max_drawdown_days']}</span></li>
            <li><span class='title'>平均每日换手率</span><span class='value'>{stats['average_daily_turnover']}%</span></li>
        </ul>
    </div>
    {plot_excess_cumulative_ratio(results)}
    """
    strategy_capacity_html = plot_strategy_capacity(results, volume_limit=volume_limit, frequency=frequency)
    if output == 'script':
        return T.plot_tabs([
            ('单利超额收益概况', stats_html),
            ('策略容量', strategy_capacity_html),
        ], output='script', title='策略容量分析')
    T.plot_tabs([
        ('单利超额收益概况', stats_html),
        ('策略容量', strategy_capacity_html),
    ], title='策略容量分析')

# Python 代码入口函数，input_1/2/3 对应三个输入端，data_1/2/3 对应三个输出端
def m13_run_bigquant_run(input_1, input_2, input_3):
    # 示例代码如下。在这里编写您的代码
    # df_date = input_1.read_pickle()  # 日期
    df = input_1.read_df()  # pq 数据
    
    # start_date = pd.to_datetime(df_date['start_date'])
    # end_date = pd.to_datetime(df_date['end_date'])
    
    # 后复权
    df['open'] = df['open'] * df['adj_factor']
    df['high'] = df['high'] * df['adj_factor']
    df['low'] = df['low'] * df['adj_factor']
    df['close'] = df['close'] * df['adj_factor']
    del df['adj_factor']
    
    # 清理 df
    df['date'] = pd.to_datetime(df['date'])
    # df = df[(df['date'] > start_date) & (df['date'] < end_date)]    
    df['instrument'] = df['instrument'].apply(lambda x: x[:7]+'HOF' if x[-2:] == 'SH' else x)
    df['instrument'] = df['instrument'].apply(lambda x: x[:7]+'ZOF' if x[-2:] == 'SZ' else x)
    df[df.select_dtypes(np.float64).columns] = df.select_dtypes(np.float64).astype(np.float32)
    
    # 输出准备
    df_1 = df[['instrument','date','open','high','low','close', 'volume', 'amount']]
    df_2 = df[['instrument','date','w_score','bias_cs','bias_sm','bias_ml','bias_cs_dd','bias_sm_dd','bias_ml_dd','cs_diff1','sm_diff1','ml_diff1','cs_diff2','sm_diff2','ml_diff2']]
    df_3 = {'instruments': ["['"+"','".join(set(df['instrument'].to_list()))+"']"], 'start_date': df.iloc[0]['date'], 'end_date': df.iloc[-1]['date']}
    
    data_1 = DataSource.write_df(df_1)  # 行情数据，DataFrame对象采用 write_df
    data_2 = DataSource.write_df(df_2)  # 因子数据
    data_3 = DataSource.write_pickle(df_3)  # 代码列表，list或字典等对象采用 write_pickle
    return Outputs(data_1=data_1, data_2=data_2, data_3=data_3)

# 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
def m13_post_run_bigquant_run(outputs):
    return outputs


m1 = M.cached.v3(
    run=m1_run_bigquant_run,
    post_run=m1_post_run_bigquant_run,
    input_ports='',
    params='{}',
    output_ports=''
)

m2 = M.datahub_load_postgres.v1(
    conn='postgres:Cong2021@DB009@121.41.121.43:5433',
    database='jyquant',
    generate_sql=m2_generate_sql_bigquant_run
)

m3 = M.df_to_csv.v2(
    input_1=m2.data,
    name='pq_data.csv'
)

m4 = M.cached.v3(
    input_1=m2.data,
    run=m4_run_bigquant_run,
    post_run=m4_post_run_bigquant_run,
    input_ports='',
    params='{}',
    output_ports=''
)

m5 = M.cached.v3(
    run=m5_run_bigquant_run,
    post_run=m5_post_run_bigquant_run,
    input_ports='',
    params='{}',
    output_ports=''
)

m6 = M.stock_ranker_train.v6(
    learning_algorithm='排序',
    number_of_leaves=30,
    minimum_docs_per_leaf=1000,
    number_of_trees=20,
    learning_rate=0.1,
    max_bins=1023,
    feature_fraction=1,
    data_row_fraction=1,
    plot_charts=True,
    ndcg_discount_base=1,
    m_lazy_run=False
)

m7 = M.stock_ranker_predict.v5(
    model=m6.model,
    m_lazy_run=False
)

m8 = M.strategy_style_show.v1(

)

m9 = M.strategy_turn_analysis.v1(

)

m10 = M.strategy_interval_return.v1(
    keep_n_yearly=6,
    keep_n_quarterly=8,
    keep_n_monthly=8
)

m11 = M.strategy_capacity_analyzer.v1(
    volume_limit=0.025,
    frequency='15m',
    calculate=m11_calculate_bigquant_run
)

m12 = M.strategy_income_distribution.v1(

)

m13 = M.cached.v3(
    run=m13_run_bigquant_run,
    post_run=m13_post_run_bigquant_run,
    input_ports='',
    params='{}',
    output_ports=''
)
